In [248]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import operator
from functools import reduce

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import os 
os.chdir('/Users/deepak/capstone/clutch-baller')
# os.chdir('./clutch-baller')
import practice_code as pc
import indiviual_stats as ind
# os.chdir('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [249]:
df = pd.read_csv('/Users/deepak/Desktop/NBA_Data/Every_Season_Data/[2005-2006]-combined-stats.csv')

##Added another column that shows the pts difference between both teams

In [250]:
new_df = pc.add_columns(df)

/Users/deepak/capstone/clutch-baller/practice_code.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df2['winner'] = np.where(df2.pts_difference >0, 'away', 'home')


In [251]:
new_df = pc.replace_nan(new_df)

##Remove unnecessary columns

In [252]:
col_to_remove = ['a1', 'a2', 'a3','a4','a5','h1','h2','h3','h4','h5', 'play_length', 'entered', 'left', 'possession', 'shot_distance', 'original_x', 'original_y', 'converted_x', 'converted_y', 'num', 'away', 'home', 'outof', 'opponent', 'reason', 'elapsed', 'play_id']

In [253]:
df_1 = pc.remove_col(new_df, col_to_remove)

##Remove Rows that are not needed (such as fouls, subs, timeouts etc)

In [254]:
df2 = pc.remove_junk_rows(df_1, 'event_type')

In [255]:
df2.head()

,game_id,data_set,date,period,away_score,home_score,remaining_time,team,event_type,assist,block,player,points,result,steal,type,description,pts_difference,winner,shots_made,shots_missed,total_rebound,FT_made,off_rebound,FT_missed,total_blocks,assist_count,3pt,3pt_shots,Dunk/Layup,Dunk/Layup_attempts,2pt_med/hard,med/hard_attempts,2pt_fg_made,2fg_attempts
2,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,0,2,00:11:46,PHI,shot,Allen Iverson,,Chris Webber,2,made,,Layup,Webber Layup (2 PTS) (Iverson 1 AST),-2,away,1,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1
3,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,2,2,00:11:31,MIL,shot,,,Michael Redd,2,made,,Running Jump Shot,Redd 16' Running Jump Shot (2 PTS),0,away,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,1
4,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,2,4,00:11:16,PHI,shot,Chris Webber,,Allen Iverson,2,made,,Jump Shot,Iverson 16' Jump Shot (2 PTS) (Webber 1 AST),-2,away,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,1
6,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,2,5,00:11:16,PHI,free throw,,,Allen Iverson,1,made,,Free Throw 1 of 1,Iverson Free Throw 1 of 1 (3 PTS),-3,away,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0
7,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,2,5,00:11:00,MIL,miss,,,Bobby Simmons,0,missed,,Jump Shot,MISS Simmons 14' Jump Shot,-3,away,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,1


#Make DF for all players stats

In [105]:
x = df2[df2['description'].str.contains('Shot') | df2['description'].str.contains('Fadeaway')]# - (df2['description'].str.contains('3PT')*1)



In [216]:
x = (((new_df['points'] != 3) & new_df['type'].str.contains('Shot') |(new_df['points'] != 3) & new_df['type'].str.contains('Fadeaway')*1).sum())




In [218]:
x- new_df['3pt_shots'].sum()

TypeError: must be str, not int

In [208]:
y = x[x.description.str.contains('3PT')]

In [209]:
g = y[y.points == 3]

In [162]:
y = new_df[(new_df['type'].str.contains('Layup') | new_df['type'].str.contains('Dunk') | new_df['type'].str.contains('Finger Roll'))]




In [134]:
x = new_df[((new_df['event_type']=='miss') | (new_df['event_type']=='shot'))]

In [10]:
player_stats = ind.object_to_int(df2, 'points')

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [13]:
assists = ind.assist_stats(player_stats)

In [16]:
blks = ind.blk_stats(player_stats)

In [17]:
names = ind.player_names(player_stats)

In [18]:
full_player_stats = ind.merge_all(names, blks, assists, player_stats)

In [21]:
full_player_stats = full_player_stats.drop(0)

In [28]:
player_stats_clutch = ind.object_to_int(regular_clutch, 'points')

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [29]:
assists = ind.assist_stats(player_stats_clutch)

In [30]:
blks = ind.blk_stats(player_stats_clutch)

In [31]:
names = ind.player_names(player_stats_clutch)

In [32]:
full_clutch_stats = ind.merge_all(names, blks, assists, player_stats_clutch)

##Regular Season or playoffs (using df_2)

In [24]:
regular_season, playoffs = pc.regular_or_playoffs(df2, 'data_set')

##Getting clutch games

In [25]:
regular_clutch = pc.clutch_moment(regular_season)

In [26]:
regular_clutch = pc.remove_not_imp_games(regular_clutch, 'game_id')

##Stats for all players in reg season, playoff, and clutch

In [27]:
ids = ind.add_ids(df2)

In [33]:
full_clutch_stats = ind.merge_ids(full_clutch_stats, ids)

In [35]:
full_clutch_stats.head()

[autoreload of practice_code failed: Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/anaconda3/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/anaconda3/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/deepak/capstone/clutch-baller/practice_code.py", line 46
    n

,player,total_games,shots_made,shots_missed,FT_made,FT_missed,assist_count,off_rebound,total_rebound,total_blocks,3pt,3pt_shots,Dunk/Layup,2pt_med/hard,2pt_fg_attempts,Dunk/Layup_attempts,med/hard_attempts,id
0,Aaron Williams,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,3
1,Adonal Foyle,13.0,5.0,6.0,2.0,2.0,1.0,6.0,16.0,6.0,0.0,0.0,2.0,3.0,5.0,3.0,8.0,4
2,Adrian Griffin,8.0,0.0,2.0,0.0,2.0,0.0,3.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5
3,Al Harrington,30.0,14.0,24.0,17.0,11.0,7.0,6.0,18.0,1.0,0.0,3.0,8.0,6.0,14.0,13.0,25.0,6
4,Al Jefferson,4.0,2.0,1.0,0.0,3.0,0.0,1.0,5.0,1.0,0.0,0.0,2.0,0.0,2.0,2.0,1.0,7


In [115]:
full_clutch_stats = full_clutch_stats.fillna(0)

In [144]:
full_playoff_stats = ind.merge_ids(full_playoff_stats, ids)
full_playoff_stats= full_playoff_stats.fillna(0)


In [121]:
full_reg_stats = ind.merge_ids(full_reg_stats, ids)
full_reg_stats = full_reg_stats.fillna(0)

pandas.core.series.Series

In [130]:
full_reg_stats = ind.get_per_game_stats(full_reg_stats)

In [132]:
full_clutch_stats= ind.get_per_game_stats(full_clutch_stats)

In [133]:
full_playoff_stats= ind.get_per_game_stats(full_playoff_stats)

In [140]:
full_playoff_stats = full_playoff_stats.drop(['id_x', 'id_y', 'id_x'], axis=1)

In [171]:
full_clutch_stats['total_shots'] =full_clutch_stats['shots_made']+full_clutch_stats['shots_missed']



In [177]:
full_clutch_stats['is_clutch'] = (full_clutch_stats['shooting%'] > full_clutch_stats['league_clutch_avg']) & (full_clutch_stats['total_shots']>20)




In [120]:
x = df[(df['event_type']=='end of period') & (df['period']>=4) & (df['pts_difference']!=0)]
x['winner'] = np.where(x.pts_difference >0, 'away', 'home')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [411]:
a = regular_season['player']


In [412]:
b_list = []
for name in a:
    if name not in b_list:
        b_list.append(name)
        

In [574]:
t= get_percent(regular_season, 'player', 'event_type' , b_list)